#Saudi News Article Analysis

In [1]:
from numpy.core.numeric import NaN
import pandas as pd
from glob import glob
import numpy as np 
import arabicstopwords.arabicstopwords as stp
import re
import nltk 
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
files_list = glob('news_*')

dfs = [] 
for file in files_list:
    data = pd.read_json(file) 
    dfs.append(data) 


In [34]:
ArNews_df = pd.concat(dfs, ignore_index=True)

In [14]:
#save the dataset 
ArNews_df.to_csv("/Users/AlaAlBinSaleh/Desktop/Capstone/data/ArNews_df.csv")
ArNews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30676 entries, 0 to 31029
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   source          30676 non-null  object
 1   url             30676 non-null  object
 2   date_extracted  30676 non-null  object
 3   title           30666 non-null  object
 4   author          26115 non-null  object
 5   content         30676 non-null  object
dtypes: object(6)
memory usage: 1.6+ MB


In [17]:
ArNews_df.head()

,source,url,date_extracted,title,author,content
0,3alyoum,http://www.3alyoum.com/news/local/a/صور-فاضحة-...,2015-08-08 06:30:06,صور فاضحة مع فتيات في لوس أنجلوس تحيل 3 قياديي...,,عين اليوم – متابعات\nأحالت أمانة منطقة المدين...
1,3alyoum,http://www.3alyoum.com/news/golobal/a/قتيل-وجر...,2015-08-08 06:30:11,قتيل وجرحى بهجوم مسلح على حافلة إيرانية في تركيا,,عين اليوم – متابعات\nقتل شخص وجرح 3 آخرين بهج...
2,3alyoum,http://www.3alyoum.com/news/local/a/مدنى-دومة-...,2015-08-08 06:30:16,مدنى دومة الجندل يسيطر على حريق في مزارع بعين ...,,رواف الكويكبي _ عين اليوم\n قال المتحدث الرسم...
3,3alyoum,http://www.3alyoum.com/news/pol/a/مظاهرات-واسع...,2015-08-08 06:30:19,مظاهرات واسعة في العراق ضد الفساد,,عين اليوم – بغداد\nدعا المرجع الشيعي الأعلى ف...
4,3alyoum,http://www.3alyoum.com/news/local/a/جثمان-الشه...,2015-08-08 06:30:22,جثمان الشهيد الحارثي يصل الطائف والصلاة عليه ظ...,,نايف السعدي – عين اليوم\n قال الناطق الإعلامي...


In [35]:
ArNews_df.isnull().values.any() 
#it cannot detect missing values 

False

In [36]:
#replacing blanks " " with nan: 
ArNews_df = ArNews_df.replace(r'^\s*$', np.nan, regex=True)

In [37]:
#check missing values again
ArNews_df.isnull().sum()

source               0
url                  0
date_extracted       0
title               67
author            4736
content            354
dtype: int64

In [38]:
# drop NaN Content 
ArNews_df = ArNews_df[ArNews_df.content.isnull() == False]
ArNews_df.content.isnull().any() #false 

False

In [39]:
#data cleaning  
ArNews_df['content'] = ArNews_df.content.str.strip() 
ArNews_df['content'] = ArNews_df.content.str.replace("\n"," ")
ArNews_df['content'] = ArNews_df.content.str.replace("–","")
ArNews_df['content'] = ArNews_df.content.str.replace("_ ","")
ArNews_df['content'] = ArNews_df.content.str.replace(":","")

In [41]:
ArNews_df['content']

0        عين اليوم  متابعات أحالت أمانة منطقة المدينة ا...
1        عين اليوم  متابعات قتل شخص وجرح 3 آخرين بهجوم ...
2        رواف الكويكبي عين اليوم  قال المتحدث الرسمي با...
3        عين اليوم  بغداد دعا المرجع الشيعي الأعلى في ا...
4        نايف السعدي  عين اليوم  قال الناطق الإعلامي بش...
                               ...                        
31025    سبق- جازان قلَّد قائد قوة أمن المنشآت العميد ب...
31026    سبق- الطائف قلَّد مدير شرطة محافظة الطائف العم...
31027    سبق- الطائف تماثل رئيس الرقباء بقاعدة الملك فه...
31028    سبق - بيشة احتفل طراد سعيد الأكلبي بزواجه أول ...
31029    فهد العتيبي- سبق- الطائف دشنت لجنة التنمية الا...
Name: content, Length: 30676, dtype: object

In [51]:
ArNews_df['content'] = ArNews_df.content.str.replace("سبق-"," ")
#ArNews_df['content']

0        عين اليوم  متابعات أحالت أمانة منطقة المدينة ا...
1        عين اليوم  متابعات قتل شخص وجرح 3 آخرين بهجوم ...
2        رواف الكويكبي عين اليوم  قال المتحدث الرسمي با...
3        عين اليوم  بغداد دعا المرجع الشيعي الأعلى في ا...
4        نايف السعدي  عين اليوم  قال الناطق الإعلامي بش...
                               ...                        
31025      جازان قلَّد قائد قوة أمن المنشآت العميد بحري...
31026      الطائف قلَّد مدير شرطة محافظة الطائف العميد ...
31027      الطائف تماثل رئيس الرقباء بقاعدة الملك فهد ا...
31028    سبق - بيشة احتفل طراد سعيد الأكلبي بزواجه أول ...
31029    فهد العتيبي-   الطائف دشنت لجنة التنمية الاجتم...
Name: content, Length: 30676, dtype: object

In [52]:
ArNews_df['content'] = ArNews_df.content.str.replace("سبق -"," ")
#ArNews_df['content']

0        عين اليوم  متابعات أحالت أمانة منطقة المدينة ا...
1        عين اليوم  متابعات قتل شخص وجرح 3 آخرين بهجوم ...
2        رواف الكويكبي عين اليوم  قال المتحدث الرسمي با...
3        عين اليوم  بغداد دعا المرجع الشيعي الأعلى في ا...
4        نايف السعدي  عين اليوم  قال الناطق الإعلامي بش...
                               ...                        
31025      جازان قلَّد قائد قوة أمن المنشآت العميد بحري...
31026      الطائف قلَّد مدير شرطة محافظة الطائف العميد ...
31027      الطائف تماثل رئيس الرقباء بقاعدة الملك فهد ا...
31028      بيشة احتفل طراد سعيد الأكلبي بزواجه أول أمس ...
31029    فهد العتيبي-   الطائف دشنت لجنة التنمية الاجتم...
Name: content, Length: 30676, dtype: object

In [27]:
#sentence segmentation 
ArNews_df['tokenized_sent'] = ArNews_df.content.str.split(".")

In [ ]:
#tokenization by words 
ArNews_df['tokenized_words'] =  ArNews_df["content"].apply(nltk.word_tokenize)